In [10]:
%load_ext autoreload
%autoreload 2

from jupyter_cadquery import set_defaults, show, Camera, Collapse
from build123d import *
import asyncio
import math
from functools import reduce

set_defaults(
	viewer="Sidecar",
	edge_accuracy=0.0001,
	ticks=25,
	grid=[True, True, True],
	cad_width=780,
	axes=True,
	axes0=True,
	glass=True,
	orbit_control=True,
	reset_camera=Camera.CENTER,
	collapse=Collapse.LEAVES
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
COLUMN_DIAM = 20
BUMP_SIZE = 0.4
COLUMN_HEIGHT = 68
LIP_THICKNESS = 2

NUM_BUMPS = 32
BASE_HEIGHT = 15
EMBOSS_THICKNESS = 0.2
FILLET_RADIUS = 1

outer_diam = COLUMN_DIAM + BUMP_SIZE * 2
column_radius = COLUMN_DIAM / 2
cap_height = outer_diam / 2
platform_diff = outer_diam - COLUMN_DIAM
base_diam = outer_diam + platform_diff

platform_width = outer_diam
length = base_diam

platform_height = LIP_THICKNESS * 4
c_height = COLUMN_HEIGHT - ((BASE_HEIGHT + LIP_THICKNESS) + (cap_height + platform_height))


In [12]:
async def make_square(length, radius=FILLET_RADIUS):
    return RectangleRounded(length, length, radius)

async def make_column_profile(column_radius, num_bumps, bump_size):
	side_length = column_radius * math.sin(math.pi / (num_bumps * 2))

	bump_profile = Rot(0,0, -90 - 90 / num_bumps) * Pos(side_length, 0, 0) * EllipticalCenterArc(
        (0,0,0),
        side_length,
        bump_size,
        end_angle=180
    )
	all_polar_locations = [v for v in PolarLocations(column_radius, count = num_bumps * 2)]
	bump_locations, arc_locations = (
        [v for i, v in enumerate(all_polar_locations) if not i % 2],
        [v for i, v in enumerate(all_polar_locations) if i % 2]
    )
	edges: ShapeList[Edge] = ShapeList([v * bump_profile for v in bump_locations]) + ShapeList([
		RadiusArc(
            v2.position,
            v1.position,
            column_radius
        ) for v1, v2 in zip(
            bump_locations,
            arc_locations
        )
	])
	return Rot(0, 0, 90 / num_bumps) * -Face(
		reduce(
			lambda x, y: x + y,
			sorted(
                edges,
                key=lambda x: x.position_at(0.5).normalized().get_signed_angle(
                    Vector(1, 0, 0)
                )
            )
		)
	)

async def make_circle(radius, radius2=None):
	if radius2 is not None:
		return Ellipse(radius, radius2)
	return Circle(radius)


async def make_base(
		base_height,
		lip_thickness,
		column_radius,
		bottom_length,
		platform_length,
		plinth_height, # default: base_height + lip_thickness
		bump_size,
		fillet_radius
	):
	r1, r2, r3, plinth_profile = await asyncio.gather(
		make_square(bottom_length, fillet_radius),
		make_square(column_radius * 2, fillet_radius),
		make_square(platform_length, fillet_radius),
		make_circle(column_radius)
	)

	base_inset = (bottom_length / 2 - column_radius)

	return loft([
		r1,
		Plane.XY.offset(lip_thickness) * r1,
		Plane.XY.offset(lip_thickness + bump_size) * r2,
		Plane.XY.offset(base_height / 2) * r2,
		Plane.XY.offset(base_height - (lip_thickness + bump_size * 2)) * r2,
		Plane.XY.offset(base_height - lip_thickness) * r3,
		Plane.XY.offset(base_height) * r3,
		Plane.XY.offset(base_height) * plinth_profile,
		Plane.XY.offset(plinth_height) * plinth_profile,
	], ruled=True)

async def make_shaft(
		shaft_radius,
		shaft_height,
		plinth_height,
		num_bumps,
		bump_size
	):
	profile, c1, c2 = await asyncio.gather(
		make_column_profile(shaft_radius, num_bumps, bump_size),
		make_circle(shaft_radius),
		make_circle(shaft_radius + bump_size)
	)
	base = loft([
		c2,
		Plane.XY.offset(bump_size) * c1
	])

	shaft = extrude(profile, shaft_height) + base + Plane.XY.offset(shaft_height) * Rot(180, 0, 0) * base
	return Plane.XY.offset(plinth_height) * shaft

def make_spiral(num_turns, dist):
	r = lambda theta: (dist / num_turns) * theta / math.pi
	angles = [-x * math.pi / 4 for x in range(num_turns * 8 + 1)]
	pts = [(r(x) * math.cos(x), r(x) * math.sin(x)) for x in angles]
	return Spline(
		pts
	)

async def make_cap(platform_length, platform_width, lip_thickness, emboss_thickness):
	platform_height = lip_thickness * 4
	roll = Cylinder(
        platform_height / 2,
        platform_length,
        rotation=(-90, 0, 0),
        align=(Align.CENTER, Align.MAX, Align.CENTER)
    )
	platform = Pos(-platform_width / 2, 0, 0) * roll + Pos(platform_width / 2, 0, 0) * roll + Box(
        platform_width,
        platform_length,
        platform_height / 2,
        align=(Align.CENTER, Align.CENTER, Align.MIN)
    )
	platform_base = Ellipse(platform_width / 2, platform_length / 2)
	ellipcylinder = Pos(0,0,platform_height / 2) * extrude(platform_base, platform_height / 2)
	platform += ellipcylinder

	spiral = Line(
		(0, platform_height / 4),
		(platform_width / 2, platform_height / 4)
	) + Pos(
        platform_width / 2,
        platform_height / 2
    ) * Rot(0, 0, 90) * make_spiral(
        1,
        platform_height / 8
    )
	spiral += spiral.mirror(Plane.YZ)

	platform += Rot(90, 0, 0) * extrude(
        -Face(spiral.offset_2d(lip_thickness / 4)),
        platform_length / 2 + emboss_thickness,
        both=True
    )
	c1 = await make_circle(platform_length / 2)
	l = loft([
		Plane.XY.offset(platform_height) * platform_base,
		Plane.XY.offset(platform_height + lip_thickness) * c1,
	], ruled=True)

	return platform + l

async def make_column():
	cap_step = LIP_THICKNESS #* 1.5
	cap_height = cap_step * 5
	base, shaft, cap = await asyncio.gather(
		make_base(
			base_height=BASE_HEIGHT,
			lip_thickness=LIP_THICKNESS,
			column_radius=COLUMN_DIAM / 2,
			bottom_length=COLUMN_DIAM + BUMP_SIZE * 2,
			platform_length=COLUMN_DIAM + BUMP_SIZE * 4,
			plinth_height=BASE_HEIGHT + LIP_THICKNESS,
			bump_size=BUMP_SIZE,
			fillet_radius=FILLET_RADIUS
		),
		make_shaft(
			COLUMN_DIAM / 2 - BUMP_SIZE,
			COLUMN_HEIGHT - ((BASE_HEIGHT + LIP_THICKNESS) + (cap_height)),
			BASE_HEIGHT + LIP_THICKNESS,
			NUM_BUMPS,
			BUMP_SIZE
		),
		make_cap(
			COLUMN_DIAM,
			COLUMN_DIAM + BUMP_SIZE,
			cap_step,
			EMBOSS_THICKNESS,
		)
	)
	
	return reduce(lambda x, y: x + y, [
        base,
        shaft,
        (Plane.XY.offset(COLUMN_HEIGHT) * Rot(180, 0, 0) * cap)
    ])
	# return [base, shaft, (Plane.XY.offset(COLUMN_HEIGHT) * Rot(180, 0, 0) * cap)]
	# return base

column = await make_column()
show(column)

+
